# Training the Fully-Connected Neural Network Model

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import h5py
import re
import pandas as pd

In [ ]:
%load_ext tensorboard
%load_ext autoreload

Load code for this project

In [ ]:
import fcnn.train
import fcnn.eval
import data_processing.data as dp
%autoreload 1
%aimport fcnn.train
%aimport fcnn.eval

### Load the training data

In [ ]:
path_data = './data_processing/voxels/'
train, _ = dp.load_discretized_data(path_data, prefix='Grid20', categorical=False, binary=True, normalize=True)

In [ ]:
train = (train[0].toarray(), train[1])

Run the following to clear the logs in the tensorboard.

In [ ]:
!rm -rf fcnn/logs/*

## Build and Train Model 

The implementation of the FCNN model is generic in a sense that one can choose the number of hidden layers, number of neurons in each layer as well as if one wants to apply dropout and at which probability. In the training, only one hidden layer was considered. An excerpt from the implemented code may be seen below:

```python
model = tf.keras.Sequential()
model.add(tf.keras.layers.Dense(hidden_layers[0], input_dim=train[FEATURES].shape[1], activation='relu'))
if use_dropout:
    model.add(tf.keras.layers.Dropout(dropout))
for neurons in hidden_layers[1:]:
    model.add(tf.keras.layers.Dense(neurons, input_dim=train[FEATURES].shape[1], activation='relu'))
    if use_dropout:
        model.add(tf.keras.layers.Dropout(dropout))
model.add(tf.keras.layers.Dense(num_categories, activation='sigmoid'))
```

**See also**: [fcnn/train.py](./fcnn/train.py)

The following strategy was applied when training the model:

## A 'simple' start:

* A subset of the data was only included in the training, e.g. 160 samples
* No regularization, i.e. dropout not activated
* A limited number of neurons were included, e.g. 32
* Started with a learning rate of 1e-5, taken from [Kuchera, 2019](https://www.sciencedirect.com/science/article/pii/S0168900219308046?via%3Dihub)
* I strived to just be able to train the model, i.e. observe a decreasing loss function with the number of epochs

In [ ]:
%%time
fcnn.train.train(train=train, 
                log_dir='fcnn/logs/',
                hidden_layers=[32],
                validation_split=0.15,
                lr=1e-5, 
                decay=0.,
                examples_limit=-160,
                epochs=20, 
                batch_size=32,
                seed=71,
                use_dropout=False,
                dropout=0.5,
               )

In [ ]:
%tensorboard --logdir fcnn/logs/ --port 6008

There was really no issue training the model (loss was steadily decreasing), see tensorboard above, therefore a further advanced model was trained:

## Towards final model:

* All data included
* No regularization, i.e. dropout not activated
* 128 neurons were included
* A faster learning rate of 1e-3 (tuned)
* Now striving to train the model smoothly, tuning the learning rate

In [ ]:
%%time
fcnn.train.train(train=train, 
                log_dir='fcnn/logs/',
                hidden_layers=[128],
                validation_split=0.15,
                lr=1e-3, 
                decay=0.,
                examples_limit=-1,
                epochs=20, 
                batch_size=32,
                seed=71,
                use_dropout=False,
                dropout=0.5,
               )

In [ ]:
%tensorboard --logdir fcnn/logs/ --port 6008

There was again no issue training the model, see tensorboard above. The model converged very fast and since the loss of the validation function does not increase after a while, there should be limited overfitting.

## Final model:

* All data included
* Dropout activated
* 128 neurons were included
* A learning rate of 1e-3 (tuned)
* Running only 12 epochs

In [ ]:
%%time
fcnn.train.train(train=train, 
                log_dir='fcnn/logs/',
                hidden_layers=[128],
                validation_split=0.15,
                lr=1e-3, 
                decay=0.,
                examples_limit=-1,
                epochs=12, 
                batch_size=32,
                seed=71,
                use_dropout=True,
                dropout=0.5,
               )

In [ ]:
%tensorboard --logdir fcnn/logs/ --port 6008